In [1]:
# 导入必要的库
import mediapipe as mp  # MediaPipe 库用于进行手部检测
import cv2  # OpenCV 库用于处理视频流和图像操作
import numpy as np  # NumPy 库用于进行数值计算
from mediapipe.framework.formats import landmark_pb2  # 导入 MediaPipe 的坐标格式模块
import time  # 时间库，用于测量时间间隔
from math import sqrt  # 数学库，用于计算平方根
from pyfirmata import Arduino, SERVO, util  # PyFirmata 库用于控制 Arduino
from time import sleep  # 导入 sleep 函数用于延时

# 定义 Arduino 端口和舵机针脚
port = "com7"  # Arduino 所连接的端口
pin = 10  # 舵机连接的针脚编号
board = Arduino(port)  # 初始化 Arduino 板
board.digital[pin].mode = SERVO  # 将指定针脚设为舵机模式

# 定义旋转舵机的函数
def rotateservo(pin, angle):
    """
    控制舵机旋转到指定角度。

    :param pin: 舵机连接的针脚
    :param angle: 旋转的角度
    """
    board.digital[pin].write(angle)  # 控制舵机旋转到指定角度
    sleep(0.015)  # 延时 15 毫秒

# 初始化 MediaPipe 的手部检测和绘图模块
mp_drawing = mp.solutions.drawing_utils  # 用于绘制手部关键点的工具
mp_hands = mp.solutions.hands  # 手部检测模块

# 打开摄像头
video = cv2.VideoCapture(0)  # 初始化摄像头，参数 0 表示默认摄像头

# 设置手部检测的置信度
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8) as hands:
    click = 0  # 初始化点击计数器

    # 循环读取摄像头数据
    while video.isOpened():
        ret, frame = video.read()  # 从摄像头获取一帧图像
        if not ret:
            break  # 如果没有读取到帧，则退出循环
        
        # 转换图像为 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 将图像从 BGR 转换为 RGB 格式
        image = cv2.flip(image, 1)  # 水平翻转图像
        imageHeight, imageWidth, _ = image.shape  # 获取图像的高度和宽度
        
        # 处理图像以检测手部
        results = hands.process(image)  # 使用 MediaPipe 处理图像以获取手部位置
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # 将图像转换回 BGR 格式以便显示

        # 检测到手部
        if results.multi_hand_landmarks:
            # 遍历检测到的每个手部
            for num, hand in enumerate(results.multi_hand_landmarks):
                # 绘制手部关键点和连接线
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                          mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))

        # 检查是否检测到多只手
        if results.multi_hand_landmarks is not None:
            # 遍历检测到的每只手
            for handLandmarks in results.multi_hand_landmarks:
                # 遍历手部的每个关键点
                for point in mp_hands.HandLandmark:
                    # 获取归一化的关键点坐标
                    normalizedLandmark = handLandmarks.landmark[point]
                    # 将归一化坐标转换为像素坐标
                    pixelCoordinatesLandmark = mp_drawing._normalized_to_pixel_coordinates(normalizedLandmark.x, 
                                                                                           normalizedLandmark.y,
                                                                                           imageWidth, 
                                                                                           imageHeight)

                    # 将关键点转为字符串
                    point = str(point)

                    # 检查关键点是否为食指尖端
                    if point == 'HandLandmark.INDEX_FINGER_TIP':
                        try:
                            indexFingerTip_x = pixelCoordinatesLandmark[0]  # 获取食指尖端的 X 坐标
                            indexFingerTip_y = pixelCoordinatesLandmark[1]  # 获取食指尖端的 Y 坐标
                        except:
                            pass
                    # 检查关键点是否为拇指尖端
                    elif point == 'HandLandmark.THUMB_TIP':
                        try:
                            thumbFingerTip_x = pixelCoordinatesLandmark[0]  # 获取拇指尖端的 X 坐标
                            thumbFingerTip_y = pixelCoordinatesLandmark[1]  # 获取拇指尖端的 Y 坐标
                        except:
                            pass

                try:
                    # 计算食指和拇指尖端之间的距离
                    Distance_x = sqrt((indexFingerTip_x - thumbFingerTip_x) ** 2 + (indexFingerTip_y - thumbFingerTip_y) ** 2)
                    Distance_y = sqrt((indexFingerTip_y - thumbFingerTip_y) ** 2 + (indexFingerTip_y - thumbFingerTip_y) ** 2)

                    # 检查距离是否小于一定阈值
                    if Distance_x < 7 or Distance_x < -7:
                        if Distance_y < 7 or Distance_y < -7:
                            click = click + 1  # 增加点击计数

                            # 每第六次检测到的点击触发一次舵机旋转
                            if click % 6 == 0:
                                print("single click")  # 打印单击消息
                                for i in range(0, 180):  # 舵机从 0 度旋转到 180 度
                                    rotateservo(pin, i)  # 调用函数旋转舵机
                                break
                except:
                    pass

        # 显示图像窗口
        cv2.imshow('Hand Tracking', image)

        # 检查是否按下 'q' 键以退出
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# 释放视频流
video.release()
cv2.destroyAllWindows()  # 销毁所有窗口


SerialException: could not open port 'com7': FileNotFoundError(2, '系统找不到指定的文件。', None, 2)